In [ ]:
import pandas as pd

from tb_incubator.constants import set_project_base_path
from tb_incubator.input import load_param_info
from tb_incubator.calibrate import get_bcm, tabulate_calib_results, plot_posterior_comparison
from tb_incubator.plotting import display_plot

import arviz as az
az.style.use("arviz-doc")

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]


In [ ]:
params= load_param_info()["value"]
param_info = load_param_info()
covid_effects = {
    'detection_reduction':True,
    'post_covid_improvement': True
}
bcm = get_bcm(params, xpert_sensitivity=True, covid_effects=covid_effects)

In [ ]:
file_suffix = "p14_134"
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')

### Parameter traces

In [ ]:
#az.plot_trace(idata, figsize=(15, 22), compact=False, legend=False)

### Comparison of parameter posteriors to priors

In [ ]:
plot_posterior_comparison(idata, 0.995)

### Table of parameter distributions

In [ ]:
from IPython.display import Markdown

results_table = tabulate_calib_results(idata, param_info)
Markdown(results_table.to_markdown())